## Import libraries and define functions

In [8]:
# PATH = "C:/Users/amd/OneDrive - National University of Singapore/SleepData"
# EDF_FILE_PATH = "C:/Users/amd/OneDrive - National University of Singapore/SleepData/szu_hospital/PSG/2024-6-20jiangyifan.edf"
# PSG_FILE_PATH = "C:/Users/amd/OneDrive - National University of Singapore/SleepData/szu_hospital/PSG/2024-6-20jiangyifan.edf"
# XML_FILE_PATH = "psg/20240620江逸凡.edf.XML"
# SAVE_PATH = "C:/Users/amd/OneDrive - National University of Singapore/SleepData/szu_hospital/PSG/merged_2024-6-20jiangyifan.edf.pkl"
# print(PSG_FILE_PATH)

In [9]:
# Load NeuroKit and other useful packages
import neurokit2 as nk
import pandas as pd
import numpy as np
import scipy.signal as signal
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from cw_radar import *
from psg import *
import importlib
import constants
import os
importlib.reload(constants)
from constants import PSG_FILE_PATH, XML_FILE_PATH, SAVE_PATH, RADAR_FILE_PATH, FULL_SAVE_PATH

def one_hot_encode(df_psg, df_xml, encode_type='stage'):
    """
    One-hot encode events or sleep stages and integrate into the PSG DataFrame.

    Parameters:
    - df_psg: DataFrame containing the PSG data.
    - df_xml: XMLProcessor object containing event and sleep stage data.
    - encode_type: str, either 'stage' or 'event' to specify what to encode.

    Returns:
    - DataFrame: PSG DataFrame with one-hot encoded columns integrated.
    """
    if encode_type == 'stage':
        df_data = df_xml.sleep_stages
        time_window = 30
        unique_items = df_data['Sleep Stage'].unique()
        time_column = 'Start Time'
    elif encode_type == 'event':
        df_data = df_xml.events
        time_window = 1
        unique_items = df_data['Name'].unique()
        time_column = 'Start'
    else:
        raise ValueError("Invalid encode_type. Must be 'stage' or 'event'.")

    # Create an empty DataFrame with the same index as df_psg
    one_hot_df = pd.DataFrame(index=df_psg.index)
    time_window = timedelta(seconds=time_window)
    
    # Create one-hot encoded columns for each unique item
    for item in unique_items:
        one_hot_df[item] = 0

    # Fill in the one-hot encoded values based on times
    for _, row in df_data.iterrows():
        start_time = row[time_column]
        
        if encode_type == 'stage':
            end_time = start_time + time_window
            item = row['Sleep Stage']
            mask = (df_psg.index >= start_time) & (df_psg.index < end_time)

        else:  # encode_type == 'event'
            end_time = row['End']
            item = row['Name']
            mask = (df_psg.index >= start_time - time_window) & (df_psg.index < end_time + time_window) # label window: 1s before and after the event
        
        one_hot_df.loc[mask, item] = 1

    return one_hot_df

def integrate(df_psg, df_xml):
    # One-hot encode events and sleep stages
    events_one_hot = one_hot_encode(df_psg, df_xml, encode_type='event')
    stages_one_hot = one_hot_encode(df_psg, df_xml, encode_type='stage')

    # Integrate the one-hot encoded DataFrames into the PSG DataFrame
    return pd.concat([df_psg, stages_one_hot, events_one_hot], axis=1)


/opt/data/private/ZhouWenren/SleepLab/psg/2024-6-20jiangyifan.edf


In [10]:
def lowpass_filter(fs, sig, highcut, order=4):
    nyq = 0.5 * fs
    high = highcut / nyq
    b, a = signal.butter(order, high, btype='lowpass')
    filtered_signal = signal.filtfilt(b, a, sig)
    return filtered_signal

def align_to_common_time_grid(psg_resampled, radar_resampled, freq_hz=64):
    """
    Aligns two resampled dataframes to a common time grid.

    Parameters:
    psg_resampled (pd.DataFrame): The first resampled dataframe.
    radar_resampled (pd.DataFrame): The second resampled dataframe.
    freq_hz (int): The frequency for the common time grid in Hz. Default is 64Hz.

    Returns:
    pd.DataFrame, pd.DataFrame: The aligned dataframes.
    """
    # Convert frequency from Hz to milliseconds
    freq_ms = f'{1000 / freq_hz}ms'

    # Define a common time grid (using the minimum and maximum timestamps)
    common_time_index = pd.date_range(
        start=max(psg_resampled.index.min(), radar_resampled.index.min()), 
        end=min(psg_resampled.index.max(), radar_resampled.index.max()), 
        freq=freq_ms  # Adjust frequency as necessary
    )

    # Reindex and interpolate both dataframes to this common time grid
    psg_aligned = psg_resampled.reindex(common_time_index, method='nearest')
    radar_aligned = radar_resampled.reindex(common_time_index, method='nearest')

    return psg_aligned, radar_aligned

def segment_and_label(df, segment_sec=10, freq_hz=64):
    """
    Segments and labels the given DataFrame based on given time window.
    Parameters:
    - df (pandas.DataFrame): The DataFrame containing the data to be segmented and labeled.
    - segment_sec (int): The duration of each segment in seconds. Default is 10.
    - freq_hz (int): The frequency of the data in Hz. Default is 64.
    Returns:
    - segmented_df (pandas.DataFrame): The segmented and labeled DataFrame.
    """
    # Initialize list to store segmented data
    segmented_data = []
    segment_size = segment_sec * freq_hz
    
    # Define columns for sleep stages and sleep events
    sleep_stages = ['Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 'REM Sleep', 
                    'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)']
    sleep_events = ['Mixed Apnea', 'Limb Movement (Left)', 'Limb Movement (Right)', 'SpO2 desaturation', 
                    'Hypopnea', 'PLM (Left)', 'PLM (Right)', 'SpO2 artifact', 'Arousal (ARO SPONT)', 
                    'Arousal (ARO PLM)', 'Arousal (ARO Limb)', 'Arousal (ARO RES)', 'Central Apnea', 
                    'Obstructive Apnea']
    
    # Loop over the DataFrame in chunks of segment_size
    for start in range(0, len(df), segment_size):
        # Get current segment
        segment = df.iloc[start:start + segment_size]
        
        # Calculate mean of continuous signals
        continuous_data = segment[['processed_signal', 'ECG', 'Thor', 'Abdo', 'SpO2']].mean().to_dict()
        
        # Determine the dominant sleep stage (most frequent one)
        sleep_stage_counts = segment[sleep_stages].sum()
        dominant_sleep_stage = sleep_stage_counts.idxmax() if sleep_stage_counts.max() > 0 else 'No Stage'
        
        # Determine the dominant sleep event (most frequent one)
        sleep_event_counts = segment[sleep_events].sum()
        dominant_sleep_event = sleep_event_counts.idxmax() if sleep_event_counts.max() > 0 else 'No Event'
        
        # Get Start Time and End Time, trimmed to whole seconds
        start_time = segment.index[0].floor('s')
        end_time = segment.index[-1].floor('s')
        
        # Compile the data for this segment
        segment_data = {'Start Time': start_time, 'End Time': end_time, **continuous_data, 
                        'Dominant Sleep Stage': dominant_sleep_stage, 'Dominant Sleep Event': dominant_sleep_event}
        segmented_data.append(segment_data)
    
    # Convert segmented data to DataFrame
    segmented_df = pd.DataFrame(segmented_data)
    return segmented_df




## Load PSG data

In [ ]:
psg_file_path = PSG_FILE_PATH
# psg_file_path = r"C:\Users\amd\OneDrive - National University of Singapore\SleepData\szu_hospital\PSG\2024-6-20jiangyifan.edf"
psg_processor = PSGDataProcessor(psg_file_path)
psg_processor.load_data()

In [ ]:
print(len(psg_processor.data['ECG']))
print(psg_processor.data['ECG'][0]) # ECG data
print(len(psg_processor.data['ECG'][0][0]))
print(psg_processor.data['ECG'][1]) # ECG timestamps
print(len(psg_processor.data['ECG'][1]))

### Extract segments of ECG data

In [ ]:
# Specify the start and stop times in seconds
tmin, tmax = 10, 20  # Extract data between 10 and 20 seconds
psg_time_segment = psg_processor.extract_data_by_range(tmin, tmax)
print(psg_time_segment)

In [ ]:
# Extract data between two timestamps
start_datetime = datetime(2024, 6, 20, 22, 10, 33) # Replace with your actual start datetime
end_datetime = datetime(2024, 6, 20, 22, 11, 33)  # Replace with your actual end datetime
data_types = ['ECG', 'Thor']  # Replace with your actual data types

print(f"Start Timestamp: {start_datetime}, End Timestamp: {end_datetime}")  # Print the start and end timestamps of the extracted data
psg_date_segment = psg_processor.extract_segment_by_timestamp(start_datetime, end_datetime, data_types)
print(psg_date_segment)


## Load XML data

In [ ]:
xml_file_path = XML_FILE_PATH
start_datetime_str = "2024-06-20 22:02:34"
xml_processor = XMLProcessor(xml_file_path, start_datetime_str)

xml_processor.load()

In [ ]:
xml_processor.sleep_stages

In [ ]:
xml_processor.events

### Unique events and sleep stages

In [ ]:
unique_event_names = xml_processor.events['Name'].unique()
print(unique_event_names)

unique_sleep_stages = xml_processor.sleep_stages['Sleep Stage'].unique()
print(unique_sleep_stages)

### Filter by event type

In [ ]:
# Assuming xml_processor.df_events is your DataFrame
xml_filtered_by_type = xml_processor.events[xml_processor.events['Name'].str.contains('limb', case=False)]
xml_filtered_by_type

### Events filtered by time range

In [ ]:
# Filter rows where 'timestamp' is within the specified range
start_datetime = datetime(2024, 6, 20, 22, 10, 33) # Replace with your actual start datetime
end_datetime = datetime(2024, 6, 20, 22, 11, 33)  # Replace with your actual end datetime
xml_filtered_by_type = xml_filtered_by_type[(xml_filtered_by_type['Start'] >= start_datetime) & (xml_filtered_by_type['Start'] <= end_datetime)]
xml_filtered_by_type

In [ ]:
print(psg_date_segment)
print(xml_filtered_by_type)

### Events sorted by duration

In [ ]:
durations = xml_processor.events['Duration']
print(durations.describe())
xml_sorted_by_duration = xml_processor.events.sort_values(by='Duration', ascending=False)
print(xml_sorted_by_duration)

## Merge the two dataframes: PSG(bio signals) and XML(event labels)

### Load PSG and XML data

In [ ]:
# psg_file_path = r"C:\Users\amd\OneDrive - National University of Singapore\SleepData\szu_hospital\PSG\2024-6-20jiangyifan.edf"
psg_file_path = PSG_FILE_PATH
psg_processor = PSGDataProcessor(psg_file_path)
psg_processor.load_data()

xml_file_path = XML_FILE_PATH
start_datetime_str = "2024-06-20 22:02:34"
xml_processor = XMLProcessor(xml_file_path, start_datetime_str)
xml_processor.load()

df_sleep_events = xml_processor.events
print(psg_processor.start_datetime)
print(xml_processor.start_datetime)

### Extract the time range of interest

In [ ]:
print(f"Start Timestamp: {psg_processor.start_datetime}, End Timestamp: {psg_processor.end_datetime}")

# Start Timestamp: 2024-06-20 22:02:34, End Timestamp: 2024-06-21 05:54:58.999023
start_datetime = datetime(2024, 6, 20, 22, 2, 34) # Replace with your actual start datetime
end_datetime = datetime(2024, 6, 21, 5, 54, 58)  # Replace with your actual end datetime
# # Extract data between two timestamps
# start_datetime = datetime(2024, 6, 20, 22, 10, 33) # Replace with your actual start datetime
# end_datetime = datetime(2024, 6, 20, 22, 11, 33)  # Replace with your actual end datetime
data_types = ['ECG', 'Thor']  # Replace with your actual data types

print(f"Start Timestamp: {start_datetime}, End Timestamp: {end_datetime}")  # Print the start and end timestamps of the extracted data
psg_date_segment = psg_processor.extract_segment_by_timestamp(start_datetime, end_datetime, data_types)
print(psg_date_segment)

In [ ]:
# Specify the start and stop times in seconds
tmin, tmax = 0, 60  # Extract data between 10 and 20 seconds
data_types = ['ECG', 'Thor']  # Replace with your actual data types

start_dt, end_dt, psg_time_segment = psg_processor.extract_data_by_range(tmin, tmax)
psg_time_segment = psg_time_segment[data_types]
print(f"Start time: {start_dt}, End time: {end_dt}")
print(psg_time_segment)

### Merge

In [ ]:
# Assuming df_psg is your processed PSG DataFrame
df_psg = psg_date_segment
# df_psg = psg_time_segment

# One-hot encode events and sleep stages
# events_one_hot_e = one_hot_encode(df_psg, xml_processor, 'event')
# events_one_hot_s = one_hot_encode(df_psg, xml_processor, 'stage')

merged_df = integrate(df_psg, xml_processor)
print(merged_df.head())

### Examine the correctness of the one-hot encoding

In [ ]:
s = xml_processor.sleep_stages
s_filtered = s[(s['Start Time'] >= start_datetime) & (s['Start Time'] <= end_datetime)]

# Filter rows within the time range
t_filtered = merged_df.loc[start_datetime:end_datetime]

# Extract the specified columns from filtered_temp
wake1 = t_filtered[t_filtered['Wakefulness (W)'] == 1]
print(wake1.head().index[0])
wake2 = s_filtered[s_filtered['Sleep Stage'] == 'Wakefulness (W)']
print(wake2)

stage1 = t_filtered[t_filtered['NREM Sleep Stage 1 (N1)'] == 1]
print(stage1.head().index[0])
stage2 = s_filtered[s_filtered['Sleep Stage'] == 'NREM Sleep Stage 1 (N1)']
print(stage2)



### Save the complete(merged) DataFrame to a CSV/Pickle file

In [ ]:
# Define the file path and get the directory path and filename
# psg_file_path = "/Users/w.z/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/SleepData/苏州大学附属医院/PSG/2024-6-20jiangyifan.edf"
psg_file_path = PSG_FILE_PATH
dir_path, filename = os.path.split(psg_file_path)
modified_filename = 'merged_' + filename

# # Save as CSV file for portability
# save_path = os.path.join(dir_path, modified_filename + '.csv')
# merged_df.to_csv(save_path, index=False)  # Set index=False if you don't want to save the index
# print(f"CSV file saved at: {save_path}")

# Save as Pickle file for efficiency and compactness
save_path = os.path.join(dir_path, modified_filename + '.pkl')
merged_df.to_pickle(save_path)
print(f"Pickle file saved at: {save_path}")

### Load the merged DataFrame

In [ ]:
# Specify the path to the pickle file
save_path = SAVE_PATH
pickle_path = save_path # '/path/to/your/filename.pkl'
psg_date_segment = pd.read_pickle(pickle_path)

# Display the first few rows of the DataFrame
print(psg_date_segment.head())

## ERA(Event-related Analysis): Sleep Stages

### The Dataset

In [ ]:
df_sleep_stages = xml_processor.sleep_stages
df_sleep_stages


In [ ]:
codes = xml_processor.sleep_stages['Sleep Stage Code'].values
print(np.unique(codes))

stages = xml_processor.sleep_stages['Sleep Stage'].values
print(np.unique(stages))

#### Prepare the data (Time window)

In [ ]:
print(psg_processor.start_datetime)
print(psg_processor.end_datetime)

In [ ]:
# start_datetime = datetime(2024, 6, 20, 22, 40, 33) # Replace with your actual start datetime
# end_datetime = datetime(2024, 6, 20, 23, 10, 33)  # Replace with your actual end datetime
start_datetime = datetime(2024, 6, 20, 22, 40, 33) # Replace with your actual start datetime
end_datetime = datetime(2024, 6, 20, 23, 00, 33)  # Replace with your actual end datetime

start_time = start_datetime
end_time = end_datetime

filtered_df = psg_date_segment.loc[start_time:end_time]
df_subset = filtered_df[['ECG', 'Thor']]
print(df_subset.head())

#### Prepare the data (add columns)

In [ ]:
def add_sleep_stage_column(df_subset, df_sleep_stages):
    # Create a new column in df_subset to store the sleep stage code
    df_subset['Stage Code'] = None

    # Iterate over each row in df_sleep_stages
    for index, row in df_sleep_stages.iterrows():
        stage_start_time = row['Start Time']
        stage_end_time = stage_start_time + timedelta(seconds=30)

        # Find the rows in df_subset that fall within the current sleep stage interval
        mask = (df_subset.index >= stage_start_time) & (df_subset.index < stage_end_time)
        
        # Assign the sleep stage code to the corresponding rows in df_subset
        df_subset.loc[mask, 'Stage Code'] = row['Sleep Stage Code']
        # df_subset.loc[mask, 'Sleep Stage'] = row['Sleep Stage']
        # df_subset.loc[mask, 'Time (seconds)'] = row['Time (seconds)']
        # df_subset.loc[mask, 'Duration'] = row['Duration']
    
    return df_subset

# Example usage
# Apply the function to add the sleep stage column
df_subset_with_sleep_stage = add_sleep_stage_column(df_subset, df_sleep_stages)

print(df_subset_with_sleep_stage)
print(df_subset_with_sleep_stage['Stage Code'].value_counts())

In [ ]:
# psg_processor.signals_diagram(df_subset_with_sleep_stage)

### Find Events (Auto)

#### Process and plot the events

In [ ]:
# Define events based on sleep stages
# df_subset_with_sleep_stage['ECG'] = df_subset_with_sleep_stage['ECG'] *100000
# df_subset_with_sleep_stage['Thor'] = df_subset_with_sleep_stage['Thor'] *100000
events1 = nk.events_find(df_subset_with_sleep_stage['Stage Code'], threshold=0, threshold_keep='below')
print(events1)
plot = nk.events_plot(events1, df_subset_with_sleep_stage)


In [ ]:
events2 = nk.events_find(df_subset_with_sleep_stage['Stage Code'], threshold=0, threshold_keep='above')
print(events2)
plot = nk.events_plot(events2, df_subset_with_sleep_stage)


In [ ]:
print(events1)
print(events2)

In [ ]:

# Combine the arrays
combined_onset = np.concatenate((events1['onset'], events2['onset']))
combined_duration = np.concatenate((events1['duration'], events2['duration']))

# Sort the arrays based on the combined onset values
sorted_indices = np.argsort(combined_onset)
sorted_onset = combined_onset[sorted_indices]
sorted_duration = combined_duration[sorted_indices]
sorted_label = [str(i+1) for i in range(len(sorted_onset))]


# Create the merged dictionary
merged_events = {
    'onset': sorted_onset,
    'duration': sorted_duration,
    'label': sorted_label
}

print(merged_events)


In [ ]:
plot = nk.events_plot(merged_events, df_subset_with_sleep_stage)

#### Plot RSP diagram (trial)

In [ ]:
psg_processor.start_datetime

In [ ]:
extracted_types = ['Thor']
start_datetime = datetime(2024, 6, 20, 22, 58, 33) # Replace with your actual start datetime
end_datetime = datetime(2024, 6, 20, 22, 59, 33)  # Replace with your actual end datetime
extracted_data = psg_processor.extract_segment_by_timestamp(start_datetime, end_datetime, extracted_types)
rsp_signals, rsp_info = psg_processor.rsp_diagram(extracted_data['Thor'])


#### Process the Signals

In [ ]:
# Process the signal
data_clean, info = nk.bio_process(ecg=df_subset_with_sleep_stage["ECG"], 
                                  rsp=df_subset_with_sleep_stage["Thor"],
                                  keep=df_subset_with_sleep_stage["Stage Code"],  
                                  sampling_rate=1024)

# Visualize some of the channels
data_clean[["ECG_Rate", "RSP_Rate", "Stage Code"]].plot(subplots=True)

In [ ]:
x = df_subset_with_sleep_stage['Stage Code'].values

plt.figure(figsize=(6, 3))  # Set the figure size for better readability
plt.plot(x, marker='o', linestyle='-', color='b')  # Plot x with markers and lines
plt.show()  

#### Create Epochs

In [ ]:
# Build and plot epochs
epochs = nk.epochs_create(data_clean, merged_events, sampling_rate=1024, epochs_start=-30, epochs_end=30)

In [ ]:
for epoch_index, epoch in epochs.items():
    for attribute, value in epoch.items():
        print(f"  {attribute}")
    print("\n")  # Adds a newline for better readability between epochs

In [ ]:
# Iterate through epoch data
for epoch in epochs.values():
    # Plot scaled signals, "Stage Code"
    nk.signal_plot(epoch[['ECG_Rate', 'RSP_Rate']], 
                   title=epoch['Label'].values[0],  # Extract condition name
                   standardize=True)  

#### Interval Analysis

In [ ]:
plot = nk.events_plot(merged_events,data_clean[["ECG_Rate", "RSP_Rate", "Stage Code"]])

In [ ]:
print(merged_events)
epochs = nk.epochs_create(data_clean, merged_events, sampling_rate=1024, epochs_start=0, epochs_end='from_events')

In [ ]:
# Analyze
nk.ecg_intervalrelated(epochs)

In [ ]:
nk.rsp_intervalrelated(epochs)

#### Plot ECG and RSP signals

In [ ]:
plt.figure(figsize=(30, 6))  # Set figure size for better readability
plt.plot(df_subset['ECG'], label='ECG')  # Plot the ECG column
plt.title('ECG Over Time')  # Title of the plot
plt.xlabel('Index')  # X-axis label
plt.ylabel('ECG Value')  # Y-axis label
plt.legend()  # Show legend
plt.show()  # Display the plot

In [ ]:
plt.figure(figsize=(30, 6))  # Set figure size for better readability
plt.plot(df_subset['Thor'], label='RSP')  # Plot the ECG column
plt.title('RSP Over Time')  # Title of the plot
plt.xlabel('Index')  # X-axis label
plt.ylabel('RPS Value')  # Y-axis label
plt.legend()  # Show legend
plt.show()  # Display the plot

#### Feature Extraction (Auto)

In [ ]:
# df = nk.bio_analyze(epochs, sampling_rate=1024)
# df

#### Feature Extraction (Manual)

In [ ]:
df_temp = {}  # Initialize an empty dict to store the results
         
# Iterate through epochs index and data
for epoch_index, epoch in epochs.items():
    df_temp[epoch_index] = {}  # Initialize an empty dict inside of it
                            

    # Note: We will use the 100th value (corresponding to the event onset, 0s) as the baseline

    # ECG ====
    ecg_baseline = epoch["ECG_Rate"].values[100]  # Baseline
    ecg_mean = epoch["ECG_Rate"][0:4].mean()  # Mean heart rate in the 0-4 seconds
    # Store ECG in df_temp
    df_temp[epoch_index]["ECG_Rate_Mean"] = ecg_mean - ecg_baseline  # Correct for baseline

    # RSP ====
    rsp_baseline = epoch["RSP_Rate"].values[100]  # Baseline
    rsp_rate = epoch["RSP_Rate"][0:6].mean()  # Longer window for RSP that has a slower dynamic
    # Store RSP in df_temp
    df_temp[epoch_index]["RSP_Rate_Mean"] = rsp_rate - rsp_baseline  # Correct for baseline

df_temp = pd.DataFrame.from_dict(df_temp, orient="index")  # Convert to a dataframe
df_temp  # Print DataFrame

#### Plot Event-Related Features

In [ ]:
df_reset = df_temp.reset_index()
print(df_reset)

In [ ]:
import seaborn as sns
sns.boxplot(x="index", y="ECG_Rate_Mean", data=df_reset)

In [ ]:
import seaborn as sns
sns.boxplot(x="index", y="RSP_Rate_Mean", data=df_reset)

### Find Events (Manual)

In [ ]:
df_sleep_stages['Duration'] = 30
df_sleep_stages

#### Prepare the data (add columns)

In [ ]:
def add_sleep_stage_column(df_subset, df_sleep_stages):
    # Create a new column in df_subset to store the sleep stage code
    df_subset['Stage Code'] = None

    # Iterate over each row in df_sleep_stages
    for index, row in df_sleep_stages.iterrows():
        stage_start_time = row['Start Time']
        stage_end_time = stage_start_time + timedelta(seconds=30)

        # Find the rows in df_subset that fall within the current sleep stage interval
        mask = (df_subset.index >= stage_start_time) & (df_subset.index < stage_end_time)
        
        # Assign the sleep stage code to the corresponding rows in df_subset
        df_subset.loc[mask, 'Stage Code'] = row['Sleep Stage Code']
        df_subset.loc[mask, 'Sleep Stage'] = row['Sleep Stage']
        df_subset.loc[mask, 'Time (seconds)'] = row['Time (seconds)']
        df_subset.loc[mask, 'Duration'] = row['Duration']
    
    return df_subset

# Example usage
# Apply the function to add the sleep stage column
df_subset_sleep = add_sleep_stage_column(df_subset, df_sleep_stages)

print(df_subset_sleep)
print(df_subset_sleep['Stage Code'].value_counts())

#### Create Events

In [ ]:
def extract_sleep_stages(df_sleep_stages, start_datetime, end_datetime):
    
    # Calculate 'End Time' assuming each stage lasts 30 seconds
    df_sleep_stages['End Time'] = df_sleep_stages['Start Time'] + pd.Timedelta(seconds=30)
    df_subset = df_sleep_stages[(df_sleep_stages['Start Time'] >= start_datetime) & (df_sleep_stages['End Time'] <= end_datetime)]
    
    return df_subset



In [ ]:
start_datetime = datetime(2024, 6, 20, 22, 40, 33) # Replace with your actual start datetime
end_datetime = datetime(2024, 6, 20, 23, 10, 33)  # Replace with your actual end datetime

df_subset_sleep = extract_sleep_stages(df_sleep_stages, start_datetime, end_datetime)
df_subset_sleep.head()

In [ ]:
df_sleep_stages_dur = df_subset_sleep
event_onsets = df_sleep_stages_dur['Time (seconds)'].values
event_durations = df_sleep_stages_dur['Duration'].values
event_conditions = df_sleep_stages_dur['Sleep Stage'].values

events = nk.events_create(event_onsets = event_onsets,
                          event_durations = event_durations,
                          event_conditions= event_conditions)

print(events)


In [ ]:
print(df_subset)
print(df_subset_sleep['Sleep Stage Code'].value_counts())

#### Plot the events

In [ ]:
events

In [ ]:

# Plot the location of event with the signals
plot = nk.events_plot(events, df_subset)

## IRA(Interval-related Analysis)

In [ ]:
# Process the signal
data_clean, info = nk.bio_process(ecg=df_subset_with_sleep_stage["ECG"], 
                                  rsp=df_subset_with_sleep_stage["Thor"],
                                  keep=df_subset_with_sleep_stage["Stage Code"],  
                                  sampling_rate=1024)

# Visualize some of the channels
data_clean[["ECG_Rate", "RSP_Rate", "Stage Code"]].plot(subplots=True)

In [ ]:
data, info = nk.ecg_process(df_subset_with_sleep_stage["ECG"], sampling_rate=1024)
nk.ecg_intervalrelated(data, sampling_rate=1024)


In [ ]:
# epochs = nk.epochs_create(df, events=[0, 15000], sampling_rate=100, epochs_end=150)

# # Half the data
# epochs = nk.epochs_create(ecg_signals, 
#                           events=[0, 15000], 
#                           sampling_rate=100, 
#                           epochs_start=0, 
#                           epochs_end=150)

# nk.ecg_intervalrelated(epochs)

## ERA(Event-related Analysis): Events

In [ ]:
# Check which columns of the df_suset have non-zero values within the given time range and return the column names
def check_non_zero_columns(df, start_datetime, end_datetime):
    # Filter rows within the time range
    df = df.loc[start_datetime:end_datetime]
    
    # Define the list of columns to extract
    columns = df.columns
    
    # Check which columns have non-zero values
    non_zero_columns = df.columns[(df != 0).any()]
    
    return non_zero_columns

In [ ]:
events = xml_processor.events['Name'].values
print(np.unique(events))

### Limb Movement

In [ ]:
event_type = 'Limb Movement (Left)'

#### Prepare the data (Time window)

In [ ]:
# start_datetime = datetime(2024, 6, 20, 22, 40, 33) # Replace with your actual start datetime
# end_datetime = datetime(2024, 6, 20, 23, 10, 33)  # Replace with your actual end datetime
start_datetime = datetime(2024, 6, 20, 22, 8, 0) # 2024-06-20 22:08:00
end_datetime = datetime(2024, 6, 20, 22, 11, 0) # 2024-06-20 22:11:00

start_time = start_datetime
end_time = end_datetime

filtered_df = psg_date_segment.loc[start_time:end_time]
# Make a df subset with all the columns except: 
# 'ECG', 'Thor', 'Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 
# 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)'
columns_to_drop = ['Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)']
df_subset = filtered_df.drop(columns=columns_to_drop)
print(df_subset.head())

#### Find Events (Auto)

#### Process and plot the events

In [ ]:
# Example usage: 
non_zero_columns = check_non_zero_columns(psg_date_segment, start_datetime, end_datetime)
print(non_zero_columns)

In [ ]:
# Define events for specific event types (label value above threshold: 0)
# , df_subset['Limb Movement (Right)']
events3 = nk.events_find(df_subset[event_type], threshold=0, threshold_keep='above')
print(events3)
plot = nk.events_plot(events3, df_subset[['ECG', 'Thor']])


In [ ]:
events4 = nk.events_find(df_subset[event_type], threshold=0, threshold_keep='below')
print(events4)
plot = nk.events_plot(events4, df_subset[['ECG', 'Thor']])

In [ ]:
# Combine the arrays
combined_onset = np.concatenate((events3['onset'], events4['onset']))
combined_duration = np.concatenate((events3['duration'], events4['duration']))

# Sort the arrays based on the combined onset values
sorted_indices = np.argsort(combined_onset)
sorted_onset = combined_onset[sorted_indices]
sorted_duration = combined_duration[sorted_indices]
sorted_label = [str(i+1) for i in range(len(sorted_onset))]


# Create the merged dictionary
merged_events = {
    'onset': sorted_onset,
    'duration': sorted_duration,
    'label': sorted_label
}

print(merged_events)


In [ ]:
plot = nk.events_plot(merged_events, df_subset[['ECG', 'Thor']])

#### Process the Signals

In [ ]:
# Process the signal
data_clean, info = nk.bio_process(ecg=df_subset["ECG"], 
                                  rsp=df_subset["Thor"],
                                  keep=df_subset[event_type],  
                                  sampling_rate=1024)

# Visualize some of the channels
data_clean[["ECG_Rate", "RSP_Rate", event_type]].plot(subplots=True)

#### Create Epochs

In [ ]:
# Build and plot epochs
epochs = nk.epochs_create(data_clean, merged_events, sampling_rate=1024, epochs_start=-30, epochs_end=30)

In [ ]:
for epoch_index, epoch in epochs.items():
    for attribute, value in epoch.items():
        print(f"  {attribute}")
    print("\n")  # Adds a newline for better readability between epochs

In [ ]:
# Iterate through epoch data
for epoch in epochs.values():
    # Plot scaled signals, "Stage Code"
    nk.signal_plot(epoch[['ECG_Rate', 'RSP_Rate']], 
                   title=epoch['Label'].values[0],  # Extract condition name
                   standardize=True)  

#### Interval Analysis

In [ ]:
plot = nk.events_plot(merged_events,data_clean[["ECG_Rate", "RSP_Rate", event_type]])

In [ ]:
print(merged_events)
epochs = nk.epochs_create(data_clean, merged_events, sampling_rate=1024, epochs_start=0, epochs_end='from_events')

In [ ]:
epochs['1'] # less than 10s (9.65s)

In [ ]:
epochs['3'] # greater than 10s (158s)

In [ ]:
epochs['4'] # less than 10s (4.3s)

In [ ]:
# Analyze
nk.ecg_intervalrelated(epochs['3'])

In [ ]:
nk.rsp_intervalrelated(epochs['3'])

### Hypopnea

In [ ]:
event_type = 'Hypopnea'

#### The Dataset

In [ ]:
pnea_df = df_sleep_events[df_sleep_events['Name'].str.contains('pnea')]
pnea_df

#### Prepare the data (Time window)

In [ ]:
# 2024-06-20 22:14:58.200	2024-06-20 22:15:10.130
start_datetime = datetime(2024, 6, 20, 22, 13, 00)
end_datetime = datetime(2024, 6, 20, 22, 16, 00) 

start_time = start_datetime
end_time = end_datetime

filtered_df = psg_date_segment.loc[start_time:end_time]
# Make a df subset with all the columns except: 
# 'ECG', 'Thor', 'Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 
# 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)'
columns_to_drop = ['Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)']
df_subset = filtered_df.drop(columns=columns_to_drop)
print(df_subset.head())

#### Prepare the data (add columns)

#### Find Events (Auto)

#### Process and plot the events

In [ ]:
# Example usage: 
non_zero_columns = check_non_zero_columns(psg_date_segment, start_datetime, end_datetime)
print(non_zero_columns)

In [ ]:
# Define events for specific event types (label value above threshold: 0)
# , df_subset['Limb Movement (Right)']
events3 = nk.events_find(df_subset[event_type], threshold=0, threshold_keep='above')
print(events3)
plot = nk.events_plot(events3, df_subset[['ECG', 'Thor']])


In [ ]:
events4 = nk.events_find(df_subset[event_type], threshold=0, threshold_keep='below')
print(events4)
plot = nk.events_plot(events4, df_subset[['ECG', 'Thor']])

In [ ]:
# Combine the arrays
combined_onset = np.concatenate((events3['onset'], events4['onset']))
combined_duration = np.concatenate((events3['duration'], events4['duration']))

# Sort the arrays based on the combined onset values
sorted_indices = np.argsort(combined_onset)
sorted_onset = combined_onset[sorted_indices]
sorted_duration = combined_duration[sorted_indices]
sorted_label = [str(i+1) for i in range(len(sorted_onset))]


# Create the merged dictionary
merged_events = {
    'onset': sorted_onset,
    'duration': sorted_duration,
    'label': sorted_label
}

print(merged_events)


In [ ]:
plot = nk.events_plot(merged_events, df_subset[['ECG', 'Thor']])

#### Process the Signals

In [ ]:
df_subset["ECG"].describe()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(30, 6))  # Set figure size for better readability
plt.plot(df_subset['ECG'], label='ECG')  # Plot the ECG column
plt.title('ECG Over Time')  # Title of the plot
plt.xlabel('Index')  # X-axis label
plt.ylabel('ECG Value')  # Y-axis label
plt.legend()  # Show legend
plt.show()  # Display the plot

In [ ]:
plt.figure(figsize=(30, 6))  # Set figure size for better readability
plt.plot(df_subset['Thor'], label='RSP')  # Plot the ECG column
plt.title('RSP Over Time')  # Title of the plot
plt.xlabel('Index')  # X-axis label
plt.ylabel('RPS Value')  # Y-axis label
plt.legend()  # Show legend
plt.show()  # Display the plot

In [ ]:
# Process the signal
data_clean, info = nk.bio_process(
                                  ecg=df_subset["ECG"],
                                  rsp=df_subset["Thor"],
                                  keep=df_subset[event_type],  
                                  sampling_rate=1024)

# Visualize some of the channels
data_clean[["ECG_Rate", "RSP_Rate", event_type]].plot(subplots=True)

#### Create Epochs

In [ ]:
# Build and plot epochs
epochs = nk.epochs_create(data_clean, merged_events, sampling_rate=1024, epochs_start=-30, epochs_end=30)

In [ ]:
for epoch_index, epoch in epochs.items():
    for attribute, value in epoch.items():
        print(f"  {attribute}")
    print("\n")  # Adds a newline for better readability between epochs

In [ ]:
# Iterate through epoch data
for epoch in epochs.values():
    # Plot scaled signals, "Stage Code"
    nk.signal_plot(epoch[['ECG_Rate', 'RSP_Rate']], 
                   title=epoch['Label'].values[0],  # Extract condition name
                   standardize=True)


#### Interval Analysis

In [ ]:
plot = nk.events_plot(merged_events,data_clean[["ECG_Rate", "RSP_Rate", event_type]])

In [ ]:
print(merged_events)
epochs = nk.epochs_create(data_clean, merged_events, sampling_rate=1024, epochs_start=0, epochs_end='from_events')

In [ ]:
nk.rsp_intervalrelated(epochs['1'])

In [ ]:
nk.rsp_intervalrelated(epochs['2'])

In [ ]:
nk.rsp_intervalrelated(epochs['4'])

In [ ]:
nk.rsp_intervalrelated(epochs['5'])

### Central Apnea

In [ ]:
event_type = 'Central Apnea'

#### The Dataset

In [ ]:
pnea_df = df_sleep_events[df_sleep_events['Name'].str.contains(event_type)]
pnea_df

#### Prepare the data (Time window)

In [ ]:
# 2024-06-21 03:33:36.600	2024-06-21 03:33:54.830
start_datetime = datetime(2024, 6, 21, 3, 33, 00)
end_datetime = datetime(2024, 6, 21, 3, 35, 00) 

start_time = start_datetime
end_time = end_datetime

filtered_df = psg_date_segment.loc[start_time:end_time]
# Make a df subset with all the columns except: 
# 'ECG', 'Thor', 'Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 
# 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)'
columns_to_drop = ['Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)']
df_subset = filtered_df.drop(columns=columns_to_drop)
print(df_subset.head())

#### Find Events (Auto)

#### Process and plot the events

In [ ]:
# Example usage: 
non_zero_columns = check_non_zero_columns(psg_date_segment, start_datetime, end_datetime)
print(non_zero_columns)

In [ ]:
# Define events for specific event types (label value above threshold: 0)
# , df_subset['Limb Movement (Right)']
events3 = nk.events_find(df_subset[event_type], threshold=0, threshold_keep='above')
print(events3)
plot = nk.events_plot(events3, df_subset[['ECG', 'Thor']])


In [ ]:
events4 = nk.events_find(df_subset[event_type], threshold=0, threshold_keep='below')
print(events4)
plot = nk.events_plot(events4, df_subset[['ECG', 'Thor']])

In [ ]:
# Combine the arrays
combined_onset = np.concatenate((events3['onset'], events4['onset']))
combined_duration = np.concatenate((events3['duration'], events4['duration']))

# Sort the arrays based on the combined onset values
sorted_indices = np.argsort(combined_onset)
sorted_onset = combined_onset[sorted_indices]
sorted_duration = combined_duration[sorted_indices]
sorted_label = [str(i+1) for i in range(len(sorted_onset))]


# Create the merged dictionary
merged_events = {
    'onset': sorted_onset,
    'duration': sorted_duration,
    'label': sorted_label
}

print(merged_events)


In [ ]:
plot = nk.events_plot(merged_events, df_subset[['ECG', 'Thor']])

#### Process the Signals

In [ ]:
df_subset["ECG"].describe()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(30, 6))  # Set figure size for better readability
plt.plot(df_subset['ECG'], label='ECG')  # Plot the ECG column
plt.title('ECG Over Time')  # Title of the plot
plt.xlabel('Index')  # X-axis label
plt.ylabel('ECG Value')  # Y-axis label
plt.legend()  # Show legend
plt.show()  # Display the plot

In [ ]:
plt.figure(figsize=(30, 6))  # Set figure size for better readability
plt.plot(df_subset['Thor'], label='RSP')  # Plot the ECG column
plt.title('RSP Over Time')  # Title of the plot
plt.xlabel('Index')  # X-axis label
plt.ylabel('RPS Value')  # Y-axis label
plt.legend()  # Show legend
plt.show()  # Display the plot

In [ ]:
# Process the signal
data_clean, info_rsp = nk.bio_process(
                                  ecg=df_subset["ECG"],
                                  rsp=df_subset["Thor"],
                                  keep=df_subset[event_type],  
                                  sampling_rate=1024)

# Visualize some of the channels
data_clean[["ECG_Rate", "RSP_Rate", event_type]].plot(subplots=True)

#### Create Epochs

In [ ]:
# Build and plot epochs
epochs = nk.epochs_create(data_clean, merged_events, sampling_rate=1024, epochs_start=-30, epochs_end=30)

In [ ]:
for epoch_index, epoch in epochs.items():
    for attribute, value in epoch.items():
        print(f"  {attribute}")
    print("\n")  # Adds a newline for better readability between epochs

In [ ]:
# Iterate through epoch data
for epoch in epochs.values():
    # Plot scaled signals, "Stage Code"
    nk.signal_plot(epoch[['ECG_Rate', 'RSP_Rate']], 
                   title=epoch['Label'].values[0],  # Extract condition name
                   standardize=True)


#### Interval Analysis

In [ ]:
plot = nk.events_plot(merged_events,data_clean[["ECG_Rate", "RSP_Rate", event_type]])

In [ ]:
print(merged_events)
epochs = nk.epochs_create(data_clean, merged_events, sampling_rate=1024, epochs_start=0, epochs_end='from_events')

In [ ]:
nk.ecg_intervalrelated(epochs["1"])

In [ ]:
nk.rsp_intervalrelated(epochs["1"])

In [ ]:
pnea_df

In [ ]:
# 2024-06-21 03:33:36.600	2024-06-21 03:33:54.830
start_datetime = datetime(2024, 6, 21, 3, 32, 00)
end_datetime = datetime(2024, 6, 21, 3, 34, 00) 

start_time = start_datetime
end_time = end_datetime

filtered_df2 = psg_date_segment.loc[start_time:end_time]
# Make a df subset with all the columns except: 
# 'ECG', 'Thor', 'Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 
# 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)'
columns_to_drop = ['Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)']
df_subset2 = filtered_df2.drop(columns=columns_to_drop)
print(df_subset2.head())

plt.figure(figsize=(30, 6))  # Set figure size for better readability
plt.plot(df_subset2['Thor'], label='RSP')  # Plot the ECG column
plt.title('RSP Over Time')  # Title of the plot
plt.xlabel('Index')  # X-axis label
plt.ylabel('RPS Value')  # Y-axis label
plt.legend()  # Show legend
plt.show()  # Display the plot

### Obstructive Apnea

In [ ]:
event_type = 'Obstructive Apnea'

#### The Dataset

In [ ]:
pnea_df = df_sleep_events[df_sleep_events['Name'].str.contains(event_type)]
pnea_df

#### Prepare the data (Time window)

In [ ]:
# 2024-06-21 03:33:36.600	2024-06-21 03:33:54.830
start_datetime = datetime(2024, 6, 21, 4, 10, 00)
end_datetime = datetime(2024, 6, 21, 4, 11, 30) 

start_time = start_datetime
end_time = end_datetime

filtered_df = psg_date_segment.loc[start_time:end_time]
# Make a df subset with all the columns except: 
# 'ECG', 'Thor', 'Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 
# 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)'
columns_to_drop = ['Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)']
df_subset = filtered_df.drop(columns=columns_to_drop)
print(df_subset.head())

#### Find Events (Auto)

#### Process and plot the events

In [ ]:
# Example usage: 
non_zero_columns = check_non_zero_columns(psg_date_segment, start_datetime, end_datetime)
print(non_zero_columns)

In [ ]:
# Define events for specific event types (label value above threshold: 0)
# , df_subset['Limb Movement (Right)']
events3 = nk.events_find(df_subset[event_type], threshold=0, threshold_keep='above')
print(events3)
plot = nk.events_plot(events3, df_subset[['ECG', 'Thor']])


In [ ]:
events4 = nk.events_find(df_subset[event_type], threshold=0, threshold_keep='below')
print(events4)
plot = nk.events_plot(events4, df_subset[['ECG', 'Thor']])

In [ ]:
# Combine the arrays
combined_onset = np.concatenate((events3['onset'], events4['onset']))
combined_duration = np.concatenate((events3['duration'], events4['duration']))

# Sort the arrays based on the combined onset values
sorted_indices = np.argsort(combined_onset)
sorted_onset = combined_onset[sorted_indices]
sorted_duration = combined_duration[sorted_indices]
sorted_label = [str(i+1) for i in range(len(sorted_onset))]


# Create the merged dictionary
merged_events = {
    'onset': sorted_onset,
    'duration': sorted_duration,
    'label': sorted_label
}

print(merged_events)


In [ ]:
plot = nk.events_plot(merged_events, df_subset[['ECG', 'Thor']])

#### Process the Signals

In [ ]:
df_subset["ECG"].describe()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(30, 6))  # Set figure size for better readability
plt.plot(df_subset['ECG'], label='ECG')  # Plot the ECG column
plt.title('ECG Over Time')  # Title of the plot
plt.xlabel('Index')  # X-axis label
plt.ylabel('ECG Value')  # Y-axis label
plt.legend()  # Show legend
plt.show()  # Display the plot

In [ ]:
plt.figure(figsize=(30, 6))  # Set figure size for better readability
plt.plot(df_subset['Thor'], label='RSP')  # Plot the ECG column
plt.title('RSP Over Time')  # Title of the plot
plt.xlabel('Index')  # X-axis label
plt.ylabel('RPS Value')  # Y-axis label
plt.legend()  # Show legend
plt.show()  # Display the plot

In [ ]:
# Process the signal
data_clean, info_rsp = nk.bio_process(
                                  ecg=df_subset["ECG"],
                                  rsp=df_subset["Thor"],
                                  keep=df_subset[event_type],  
                                  sampling_rate=1024)

# Visualize some of the channels
data_clean[["ECG_Rate", "RSP_Rate", event_type]].plot(subplots=True)

#### Create Epochs

In [ ]:
# Build and plot epochs
epochs = nk.epochs_create(data_clean, merged_events, sampling_rate=1024, epochs_start=-10, epochs_end=10)

In [ ]:
for epoch_index, epoch in epochs.items():
    for attribute, value in epoch.items():
        print(f"  {attribute}")
    print("\n")  # Adds a newline for better readability between epochs

In [ ]:
# Iterate through epoch data
for epoch in epochs.values():
    # Plot scaled signals, "Stage Code"
    nk.signal_plot(epoch[['ECG_Rate', 'RSP_Rate']], 
                   title=epoch['Label'].values[0],  # Extract condition name
                   standardize=True)


#### Interval Analysis

In [ ]:
plot = nk.events_plot(merged_events,data_clean[["ECG_Rate", "RSP_Rate", event_type]])

In [ ]:
print(merged_events)
epochs = nk.epochs_create(data_clean, merged_events, sampling_rate=1024, epochs_start=0, epochs_end='from_events')

In [ ]:
nk.ecg_intervalrelated(epochs)

In [ ]:
nk.rsp_intervalrelated(epochs)

In [ ]:
pnea_df

In [ ]:
# 2024-06-21 03:33:36.600	2024-06-21 03:33:54.830
start_datetime = datetime(2024, 6, 21, 4, 10, 00)
end_datetime = datetime(2024, 6, 21, 4, 11, 30) 

start_time = start_datetime
end_time = end_datetime

filtered_df2 = psg_date_segment.loc[start_time:end_time]
# Make a df subset with all the columns except: 
# 'ECG', 'Thor', 'Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 
# 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)'
columns_to_drop = ['Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)']
df_subset2 = filtered_df2.drop(columns=columns_to_drop)
print(df_subset2.head())

plt.figure(figsize=(30, 6))  # Set figure size for better readability
plt.plot(df_subset2['Thor'], label='RSP')  # Plot the ECG column
plt.title('RSP Over Time')  # Title of the plot
plt.xlabel('Index')  # X-axis label
plt.ylabel('RPS Value')  # Y-axis label
plt.legend()  # Show legend
plt.show()  # Display the plot

# Test 1: Create 3D dataframe

In [ ]:
events = xml_processor.events['Name'].values
unique_events = np.unique(events)
unique_events

In [ ]:
print(psg_processor.start_datetime)
print(psg_processor.end_datetime)

In [ ]:
event_type = 'Obstructive Apnea'
event_df = df_sleep_events[df_sleep_events['Name'].str.contains(event_type)]
event_df

# 2024-06-21 03:33:36.600	2024-06-21 03:33:54.830
start_datetime = datetime(2024, 6, 20, 22, 2, 35)
end_datetime = datetime(2024, 6, 21, 5, 54, 58) 

start_time = start_datetime
end_time = end_datetime

filtered_df = psg_date_segment.loc[start_time:end_time]
# Make a df subset with all the columns except: 
# 'ECG', 'Thor', 'Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 
# 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)'
columns_to_drop = ['Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)']
df_subset = filtered_df.drop(columns=columns_to_drop)
print(df_subset.head())


In [ ]:
df_subset.columns


In [ ]:
# Initialize the dictionary to hold the 3-level structure
structured_data = {}

# Ensure the 'Start' and 'End' times in xml_processor.events are in datetime format
# Assuming 'Start' and 'End' are already in datetime or timedelta format
events_df = xml_processor.events

# Iterate over each unique event type
for event_type in events_df['Name'].unique():
    # Filter the events of this type
    event_occurrences = events_df[events_df['Name'] == event_type]
    
    # Initialize a list to hold all occurrences for this event type
    occurrences_list = []
    
    # Iterate over each occurrence of this event type
    for _, event_row in event_occurrences.iterrows():
        start_time = event_row['Start']
        end_time = event_row['End']
        
        # Extract the relevant rows from df_subset for ECG and Thor within the time range
        relevant_data = df_subset[(df_subset.index >= start_time) & (df_subset.index <= end_time)][['ECG', 'Thor']]
        
        # Add the relevant data for this occurrence
        occurrences_list.append({
            'Start': start_time,
            'End': end_time,
            'ECG': relevant_data['ECG'].values,
            'Thor': relevant_data['Thor'].values,
            'Time': relevant_data.index
        })
    
    # Store the list of occurrences for this event type
    structured_data[event_type] = occurrences_list

# Example: Access data for a specific event type
# structured_data['Mixed Apnea'] will give you the occurrences and corresponding ECG/Thor data


In [ ]:
def plot_event_data(event_type):
    # Check if the event_type exists in the structured_data
    if event_type not in structured_data:
        print(f"Event type '{event_type}' not found in structured data.")
        return
    
    # Retrieve all occurrences for the given event type
    event_data = structured_data[event_type]
    
    # Iterate over each occurrence and plot ECG and Thor data
    for i, occurrence in enumerate(event_data):
        # Create subplots for ECG and Thor
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 4))  # 2 rows, 1 column

        # Plot ECG
        ax1.plot(occurrence['Time'], occurrence['ECG'], label='ECG')
        ax1.set_title(f'{event_type} - ECG (# {i + 1})')
        ax1.set_xlabel('Time')
        ax1.set_ylabel('ECG Value')
        ax1.legend()

        # Plot Thor
        ax2.plot(occurrence['Time'], occurrence['Thor'], label='Thor')
        ax2.set_title(f'{event_type} - Thor (# {i + 1})')
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Thor Value')
        ax2.legend()

        plt.tight_layout()  # Adjust spacing between subplots
        plt.show()  # Display the plot

#### Central Apnea

In [ ]:
# Example usage:
event_type_name = "Central Apnea"  # Replace with the event type you want to plot
plot_event_data(event_type_name)


#### Obstructive Apnea

In [ ]:
# Example usage:
event_type_name = "Obstructive Apnea"  # Replace with the event type you want to plot
plot_event_data(event_type_name)

#### Hypopnea

In [ ]:
# Example usage:
event_type_name = "Hypopnea"  # Replace with the event type you want to plot
plot_event_data(event_type_name)


In [ ]:
psg_date_segment

# Test 2: Identify events at whole-night scale

In [ ]:
df_sleep_stages = xml_processor.sleep_stages

start_datetime = psg_processor.start_datetime
end_datetime = psg_processor.end_datetime

filtered_df = psg_date_segment.loc[start_datetime:end_datetime]
df_subset = filtered_df[['ECG', 'Thor']]
print(df_subset.head())

In [ ]:
def add_sleep_stage_column(df_subset, df_sleep_stages):
    # Create a new column in df_subset to store the sleep stage code
    df_subset['Stage Code'] = None

    # Iterate over each row in df_sleep_stages
    for index, row in df_sleep_stages.iterrows():
        stage_start_time = row['Start Time']
        stage_end_time = stage_start_time + timedelta(seconds=30)

        # Find the rows in df_subset that fall within the current sleep stage interval
        mask = (df_subset.index >= stage_start_time) & (df_subset.index < stage_end_time)
        
        # Assign the sleep stage code to the corresponding rows in df_subset
        df_subset.loc[mask, 'Stage Code'] = row['Sleep Stage Code']
        # df_subset.loc[mask, 'Sleep Stage'] = row['Sleep Stage']
        # df_subset.loc[mask, 'Time (seconds)'] = row['Time (seconds)']
        # df_subset.loc[mask, 'Duration'] = row['Duration']
    
    return df_subset

# Example usage
# Apply the function to add the sleep stage column
df_subset_with_sleep_stage = add_sleep_stage_column(df_subset, df_sleep_stages)

print(df_subset_with_sleep_stage)
print(df_subset_with_sleep_stage['Stage Code'].value_counts())

In [ ]:
df_subset_with_sleep_stage = df_subset_with_sleep_stage.dropna(subset=['Stage Code'])


In [ ]:
events1 = nk.events_find(df_subset_with_sleep_stage['Stage Code'], threshold=0, threshold_keep='below')
events2 = nk.events_find(df_subset_with_sleep_stage['Stage Code'], threshold=0, threshold_keep='above')


# Combine the arrays
combined_onset = np.concatenate((events1['onset'], events2['onset']))
combined_duration = np.concatenate((events1['duration'], events2['duration']))

# Sort the arrays based on the combined onset values
sorted_indices = np.argsort(combined_onset)
sorted_onset = combined_onset[sorted_indices]
sorted_duration = combined_duration[sorted_indices]
sorted_label = [str(i+1) for i in range(len(sorted_onset))]


# Create the merged dictionary
merged_events = {
    'onset': sorted_onset,
    'duration': sorted_duration,
    'label': sorted_label
}

print(merged_events)

In [ ]:
# Process the signal
data_clean, info = nk.bio_process(ecg=df_subset_with_sleep_stage["ECG"], 
                                  rsp=df_subset_with_sleep_stage["Thor"],
                                  keep=df_subset_with_sleep_stage["Stage Code"],  
                                  sampling_rate=1024)

# Visualize some of the channels
data_clean[["ECG_Rate", "RSP_Rate", "Stage Code"]].plot(subplots=True)

In [ ]:
plot = nk.events_plot(merged_events,data_clean[["ECG_Rate", "RSP_Rate", "Stage Code"]])


In [ ]:
epochs = nk.epochs_create(data_clean, merged_events, sampling_rate=1024, epochs_start=0, epochs_end='from_events')


In [ ]:
nk.rsp_intervalrelated(epochs)


In [ ]:
nk.ecg_intervalrelated(epochs)

In [ ]:
epochs = nk.epochs_create(data_clean, merged_events, sampling_rate=1024, epochs_start=-30, epochs_end=30)
# Iterate through epoch data
for epoch in epochs.values():
    # Plot scaled signals, "Stage Code"
    nk.signal_plot(epoch[['ECG_Rate', 'RSP_Rate']], 
                   title=epoch['Label'].values[0],  # Extract condition name
                   standardize=True)  

In [ ]:
print(df_subset_with_sleep_stage.head())
print(df_subset_with_sleep_stage['Stage Code'].value_counts())

In [ ]:
# Assuming df is your DataFrame and 'Stage Code' is the column you want to subset by

# Get the unique stage codes
unique_stage_codes = df_subset_with_sleep_stage['Stage Code'].unique()

# Create a dictionary to hold the subsets
stage_code_subsets = {}

# Loop through each unique stage code and generate a subset
for code in unique_stage_codes:
    # Create a subset for the current stage code and sort by 'Time'
    subset = df_subset_with_sleep_stage[df_subset_with_sleep_stage['Stage Code'] == code].sort_index(ascending=True)
    
    # Store the subset in the dictionary
    stage_code_subsets[code] = subset

# Now stage_code_subsets is a dictionary where the keys are stage codes
# and the values are the corresponding DataFrames


In [ ]:
stage_0 = stage_code_subsets[0]

In [ ]:
data_clean, info = nk.bio_process(ecg=stage_0["ECG"], 
                                  rsp=stage_0["Thor"],
                                  keep=stage_0["Stage Code"],  
                                  sampling_rate=1024)

# Visualize some of the channels
data_clean[["ECG_Rate", "RSP_Rate", "Stage Code"]].plot(subplots=True)

In [ ]:
# If df_subset has only one unique Stage Code, create an "event" that covers the entire duration
events = {'onset': [0], 'duration': [len(df_subset)], 'label': [df_subset['Stage Code'].iloc[0]]}
# Create a single epoch that covers the entire subset
epochs = nk.epochs_create(data_clean, events, sampling_rate=1024, epochs_start=0, epochs_end=len(df_subset)/1024)

# Perform the interval-related analysis
rsp_results = nk.rsp_intervalrelated(epochs)
ecg_results = nk.ecg_intervalrelated(epochs)

# Output the results
print(rsp_results)
print(ecg_results)

In [ ]:
data_clean[["ECG_Rate"]].plot(subplots=True)

In [ ]:
data_clean[["ECG_Rate"]]
mean_value = data_clean["ECG_Rate"].mean()
std_dev = data_clean["ECG_Rate"].std()
mean_value, std_dev

In [ ]:
rsp_results.keys()

In [ ]:
ecg_results.keys()

In [ ]:
print(rsp_results)


In [ ]:
data_clean[["RSP_Rate"]]

In [ ]:
import numpy as np

def calculate_sleep_stage_features(rsp_rate):
    # Ensure the input is a numpy array
    rsp_rate = np.array(rsp_rate)
    
    # 1. RSP_Rate_Mean (Mean Respiratory Rate)
    rsp_rate_mean = np.mean(rsp_rate)
    
    # 2. RRV_LFHF (Low Frequency to High Frequency Ratio)
    # Here, we will simulate LF and HF components. In practice, these would be derived from a frequency analysis
    # Using an FFT (Fast Fourier Transform) or another spectral analysis method.
    fft_values = np.fft.fft(rsp_rate)
    freqs = np.fft.fftfreq(len(rsp_rate))
    
    # Define LF (0.04 - 0.15 Hz) and HF (0.15 - 0.4 Hz) ranges
    lf_band = (0.04, 0.15)
    hf_band = (0.15, 0.4)
    
    lf_power = np.sum(np.abs(fft_values[(freqs >= lf_band[0]) & (freqs <= lf_band[1])])**2)
    hf_power = np.sum(np.abs(fft_values[(freqs >= hf_band[0]) & (freqs <= hf_band[1])])**2)
    
    rrv_lfhf = lf_power / hf_power if hf_power != 0 else np.nan
    
    # 3. RSP_RVT (Respiratory Volume per Time)
    # This is usually calculated using a more complex method involving the integration of the flow signal.
    # For simplicity, we'll assume it relates to the amplitude of RSP rate changes.
    rsp_rvt = np.std(rsp_rate)
    
    # 4. RRV_RMSSD (Root Mean Square of the Successive Differences)
    diff_rsp_rate = np.diff(rsp_rate)
    rrv_rmssd = np.sqrt(np.mean(diff_rsp_rate ** 2))
    
    # 5. RSP_Phase_Duration_Ratio (Inspiration/Expiration Duration Ratio)
    # Assuming inspiration and expiration phases are marked by increases and decreases in RSP rate,
    # We could approximate this by finding the ratio of mean positive changes to mean negative changes.
    inspiration_durations = rsp_rate[diff_rsp_rate > 0]
    expiration_durations = rsp_rate[diff_rsp_rate < 0]
    
    insp_duration_mean = np.mean(inspiration_durations) if len(inspiration_durations) > 0 else np.nan
    exp_duration_mean = np.mean(expiration_durations) if len(expiration_durations) > 0 else np.nan
    
    rsp_phase_duration_ratio = insp_duration_mean / exp_duration_mean if exp_duration_mean != 0 else np.nan
    
    # Return the features as a dictionary
    features = {
        'RSP_Rate_Mean': rsp_rate_mean,
        'RRV_LFHF': rrv_lfhf,
        'RSP_RVT': rsp_rvt,
        'RRV_RMSSD': rrv_rmssd,
        'RSP_Phase_Duration_Ratio': rsp_phase_duration_ratio
    }
    
    return features


In [ ]:
calculate_sleep_stage_features(data_clean[["RSP_Rate"]])

In [ ]:
data_clean[["RSP_Rate"]].plot(subplots=True)

In [ ]:
for epoch_index, epoch in epochs.items():
    for attribute, value in epoch.items():
        print(f"  {attribute}")
    print("\n")  # Adds a newline for better readability between epochs

# Test 3: Resample PSG data

In [ ]:
from scipy.signal import resample_poly

def resample_data(data, original_freq, target_freq):
    """
    Resamples the signal data to the target frequency.
    
    Parameters:
    - data: DataFrame or NumPy array containing the data. If DataFrame, it should have signal channels and a datetime index ('Time').
    - original_freq: The original sampling frequency (Hz).
    - target_freq: The target sampling frequency (Hz).
    
    Returns:
    - Resampled data: DataFrame or NumPy array, depending on input type.
    """
    # Determine resampling factor
    if original_freq == target_freq:
        return data  # No resampling needed

    resample_factor = original_freq / target_freq

    if isinstance(data, pd.DataFrame):
        # Resampling for DataFrame
        if original_freq > target_freq:
            # Downsampling
            resampled_data = data.apply(lambda x: resample_poly(x, up=1, down=int(resample_factor)))
        else:
            # Upsampling
            resampled_data = data.apply(lambda x: resample_poly(x, up=int(target_freq/original_freq), down=1))
        
        # # Resample the 'Time' index
        # new_time_index = pd.date_range(start=data.index[0], periods=len(resampled_data), freq=f'{1000/target_freq}ms')
        # resampled_data.index = new_time_index
        
        # Resample the 'Time' index using interpolation
        resampled_data.index = pd.date_range(start=data.index[0], end=data.index[-1], periods=len(resampled_data))


        return resampled_data

In [ ]:
plt.rcParams['figure.figsize'] = (20, 2)

In [ ]:
print(f"Start Timestamp: {psg_processor.start_datetime}, End Timestamp: {psg_processor.end_datetime}")
print(psg_processor.sampling_rate)

# Extract data between two timestamps
start_datetime = datetime(2024, 6, 20, 22, 10, 33) # Replace with your actual start datetime
end_datetime = datetime(2024, 6, 20, 22, 11, 33)  # Replace with your actual end datetime

print(f"Start Timestamp: {start_datetime}, End Timestamp: {end_datetime}")  # Print the start and end timestamps of the extracted data
psg_date_segment = psg_processor.extract_segment_by_timestamp(start_datetime, end_datetime)
psg_date_segment

In [ ]:
# Example usage
psg_resampled = resample_data(psg_date_segment, original_freq=psg_processor.sampling_rate, target_freq=64)
psg_resampled


In [ ]:
# Plot the original dataset
plt.plot(psg_date_segment.index, psg_date_segment['ECG'], label='Original')
plt.xlabel('Time')
plt.ylabel('ECG')
plt.title('Original ECG Data')
plt.legend()

# Plot the downsampled dataset
plt.plot(psg_resampled.index, psg_resampled['ECG'], label='Downsampled')
plt.xlabel('Time')
plt.ylabel('ECG')
plt.title('Downsampled ECG Data')
plt.legend()

# Show the plots
plt.show()

In [ ]:
# Plot the original dataset
plt.plot(psg_date_segment.index, psg_date_segment['Thor'], label='Original')
plt.xlabel('Time')
plt.ylabel('Thor')
plt.title('Original Thor Data')
plt.legend()

# Plot the downsampled dataset
plt.plot(psg_resampled.index, psg_resampled['Thor'], label='Downsampled')
plt.xlabel('Time')
plt.ylabel('Thor')
plt.title('Downsampled Thor Data')
plt.legend()

# Show the plots
plt.show()

# Test 4: Resample CW Radar Data

In [ ]:
from cw_radar import *
from datetime import datetime


# Example usage of CWDataProcessor
file_path = "/opt/data/private/ZhouWenren/SleepLab/cw_radar/radar20240620220948433561.csv"
radar_sample_rate = 1002

# Create an instance of the CWDataProcessor
cw_processor = CWDataProcessor(file_path, radar_sample_rate)

# Load the data from the specified file
cw_processor.load_data()

In [ ]:
d = cw_processor.data
d_time = d['timestamp'].iloc[-1].strftime('%Y-%m-%d %H:%M:%S')
print(f"End time: {d_time}")
# Start Timestamp: 2024-06-20 22:02:34, End Timestamp: 2024-06-21 05:54:58.999023


In [ ]:
data_subset = extract_data_subset(cw_processor.data, start_datetime, end_datetime)


In [ ]:
processed_sig = cw_processor.process_signal(data_subset)
radar_resampled = resample_data(processed_sig, original_freq=radar_sample_rate, target_freq=64)
radar_resampled


In [ ]:
# Plot the original dataset
plt.figure(figsize=(20, 3))
plt.plot(processed_sig, label='Original')
plt.xlabel('Time')
plt.ylabel('Radar Signal')
plt.title('Original Radar Signal')
plt.legend()

# Plot the downsampled dataset
plt.figure(figsize=(20, 3))
plt.plot(radar_resampled, label='Downsampled')
plt.xlabel('Time')
plt.ylabel('Radar Signal')
plt.title('Downsampled Radar Signal')
plt.legend()

# Show the plots
plt.show()

In [ ]:
radar_resampled['processed_signal'] = lowpass_filter(radar_sample_rate, radar_resampled['processed_signal'], 10, order=4)

In [ ]:
# Create a figure and three subplots vertically
fig, axs = plt.subplots(5, 1, figsize=(20, 10))

# Plot the downsampled ECG data
axs[0].plot(psg_resampled.index, psg_resampled['ECG'], label='Downsampled')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('ECG')
axs[0].set_title('Downsampled ECG Data')
axs[0].legend()

# Plot the downsampled Thor data
axs[1].plot(psg_resampled.index, psg_resampled['Thor'], label='Downsampled')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Thor')
axs[1].set_title('Downsampled Thor Data')
axs[1].legend()

# Plot the downsampled ECG data again
axs[2].plot(radar_resampled.index, radar_resampled['processed_signal'], label='Downsampled')
axs[2].set_xlabel('Time')
axs[2].set_ylabel('Radar Signal')
axs[2].set_title('Downsampled Radar Signal')
axs[2].legend()

# Plot the downsampled Abdo data
axs[3].plot(psg_resampled.index, psg_resampled['Abdo'], label='Downsampled')
axs[3].set_xlabel('Time')
axs[3].set_ylabel('Abdo')
axs[3].set_title('Downsampled Abdo Data')
axs[3].legend()

# Plot the downsampled SpO2 data
axs[4].plot(psg_resampled.index, psg_resampled['SpO2'], label='Downsampled')
axs[4].set_xlabel('Time')
axs[4].set_ylabel('SpO2')
axs[4].set_title('Downsampled SpO2 Data')
axs[4].legend()

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
print(psg_date_segment)
print(data_subset)

In [ ]:
print(psg_resampled)
print(radar_resampled)

In [ ]:
# Define a common time grid (using the minimum and maximum timestamps)
common_time_index = pd.date_range(
    start=max(psg_resampled.index.min(), radar_resampled.index.min()), 
    end=min(psg_resampled.index.max(), radar_resampled.index.max()), 
    freq='15.625ms'  # Adjust frequency as necessary, here it matches 64Hz
)

# Reindex and interpolate both dataframes to this common time grid
psg_aligned = psg_resampled.reindex(common_time_index, method='nearest')
radar_aligned = radar_resampled.reindex(common_time_index, method='nearest')

# Now psg_aligned and radar_aligned have the same time index and can be compared


In [ ]:
# Plot the aligned signals
plt.figure(figsize=(20, 3))
plt.plot(psg_aligned['Thor'], label='Aligned PSG Signal')
plt.plot(radar_aligned, label='Aligned Radar Signal')
# plt.plot(radar_resampled, label='Aligned & Low pass Filtered Radar Signal')
plt.xlabel('Time')
plt.ylabel('Signal Amplitude')
plt.title('Aligned Signals After Cross-Correlation')
plt.legend()
plt.grid(True)
plt.show()


## cross_correlation_time_alignment

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

# def cross_correlation_time_alignment(signal1, signal2, sample_rate):
#     """
#     Aligns two signals by calculating the cross-correlation and shifting one signal to match the other.

#     Parameters:
#     - signal1, signal2: Pandas Series with timestamps as index.
#     - sample_rate: The sample rate of the signals (in Hz).

#     Returns:
#     - aligned_signal1, aligned_signal2: Aligned signals as Pandas Series.
#     """
#     # Ensure both signals have the same length
#     min_length = min(len(signal1), len(signal2))
#     signal1 = signal1.iloc[:min_length]
#     signal2 = signal2.iloc[:min_length]

#     # Calculate cross-correlation
#     correlation = np.correlate(signal1 - np.mean(signal1), signal2 - np.mean(signal2), mode='full')
#     lag = np.argmax(correlation) - (len(signal1) - 1)
    
#     # Convert lag to time
#     time_shift = lag / sample_rate
    
#     # Shift signal1 or signal2 based on the calculated time lag
#     if lag > 0:
#         aligned_signal1 = signal1.iloc[lag:].reset_index(drop=True)
#         aligned_signal2 = signal2.iloc[:-lag].reset_index(drop=True)
#     else:
#         aligned_signal1 = signal1.iloc[:lag].reset_index(drop=True)
#         aligned_signal2 = signal2.iloc[-lag:].reset_index(drop=True)
    
#     # Create a common time index for the aligned signals
#     common_time_index = pd.date_range(
#         start=signal1.index[0] + pd.to_timedelta(time_shift, unit='s'), 
#         periods=len(aligned_signal1), 
#         freq=f'{1000 / sample_rate}ms'
#     )
    
#     aligned_signal1.index = common_time_index
#     aligned_signal2.index = common_time_index
    
#     return aligned_signal1, aligned_signal2, time_shift

# # Example usage:
# # Assuming 'psg_resampled' and 'radar_resampled' are your Pandas Series with the same time index
# psg_signal = psg_aligned['Thor']  # Replace with your actual signal column
# radar_signal = radar_aligned['processed_signal']  # Replace with your actual signal column

# aligned_psg_signal, aligned_radar_signal, time_shift = cross_correlation_time_alignment(
#     psg_signal, radar_signal, sample_rate=64  # Replace 64 with your actual sample rate
# )

# print(f"Time shift applied: {time_shift} seconds")

# # Plot the aligned signals
# plt.figure(figsize=(20, 3))
# plt.plot(aligned_psg_signal, label='Aligned PSG Signal')
# plt.plot(aligned_radar_signal, label='Aligned Radar Signal')
# # plt.plot(radar_aligned, label='Aligned Radar Signal')
# # plt.plot(radar_resampled, label='original Radar Signal')
# plt.xlabel('Time')
# plt.ylabel('Signal Amplitude')
# plt.title('Aligned Signals After Cross-Correlation')
# plt.legend()
# plt.grid(True)
# plt.show()


# Test 5: Prepare Dataframe for ML Model

In [ ]:
# Load PSG data
psg_processor = PSGDataProcessor(PSG_FILE_PATH)
psg_processor.load_data()

# Load radar data
radar_sample_rate = 1002
cw_processor = CWDataProcessor(RADAR_FILE_PATH, radar_sample_rate)
cw_processor.load_data()

# Load XML data
xml_processor = XMLProcessor(XML_FILE_PATH, psg_processor.start_datetime)
xml_processor.load()

In [ ]:
# Extract data between two timestamps
# start_datetime = datetime(2024, 6, 20, 22, 10, 33) # Replace with your actual start datetime
# end_datetime = datetime(2024, 6, 20, 22, 11, 33)  # Replace with your actual end datetime
start_datetime = psg_processor.start_datetime
end_datetime = psg_processor.end_datetime
print(f"Start Timestamp: {start_datetime}, End Timestamp: {end_datetime}")  # Print the start and end timestamps of the extracted data
psg_date_segment = psg_processor.extract_segment_by_timestamp()

radar_data_subset = extract_data_subset(cw_processor.data, start_datetime, end_datetime)
processed_sig = cw_processor.process_signal(radar_data_subset)



In [ ]:
psg_resampled = resample_data(psg_date_segment, original_freq=psg_processor.sampling_rate, target_freq=64)

radar_resampled = resample_data(processed_sig, original_freq=radar_sample_rate, target_freq=64)
radar_resampled['processed_signal'] = lowpass_filter(radar_sample_rate, radar_resampled['processed_signal'], 10, order=4)

# Reindex and interpolate both dataframes to this common time grid
psg_aligned, radar_aligned = align_to_common_time_grid(psg_resampled, radar_resampled, freq_hz=64)


In [ ]:
# Plot the aligned signals
plt.figure(figsize=(20, 3))
plt.plot(psg_aligned['Thor'], label='Aligned PSG Signal')
plt.plot(radar_aligned, label='Aligned & Low pass Filtered Radar Signal')
# plt.plot(radar_resampled, label='Low pass Filtered Radar Signal')
plt.xlabel('Time')
plt.ylabel('Signal Amplitude')
plt.title('Aligned Signals After Cross-Correlation')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
merged_aligned_df = psg_aligned.copy()
# Insert the 'processed_signal' column from radar_aligned at the specified position (0)
merged_aligned_df.insert(0, 'processed_signal', radar_aligned)

# Integrate the XML data with the merged aligned DataFrame
merged_df = integrate(merged_aligned_df, xml_processor)
print(merged_df.head())


### Save the merged DataFrame to a file

In [ ]:
# Define the file path and get the directory path and filename
psg_file_path = PSG_FILE_PATH
dir_path, filename = os.path.split(psg_file_path)
modified_filename = 'full_merged_' + filename

# Save as Pickle file for efficiency and compactness
save_path = os.path.join(dir_path, modified_filename + '.pkl')
merged_df.to_pickle(save_path)
print(f"Pickle file saved at: {save_path}")

### Load the merged data from the saved Pickle file

In [3]:
from constants import PSG_FILE_PATH, XML_FILE_PATH, SAVE_PATH, RADAR_FILE_PATH, FULL_SAVE_PATH
import pandas as pd

# Specify the path to the pickle file
save_path = FULL_SAVE_PATH
pickle_path = save_path # '/path/to/your/filename.pkl'
merged_df = pd.read_pickle(pickle_path)

# Display the first few rows of the DataFrame
print(merged_df.head())

/opt/data/private/ZhouWenren/SleepLab/psg/2024-6-20jiangyifan.edf
                            processed_signal       ECG          Thor  \
2024-06-20 22:09:48.435561          0.224244 -0.000216  1.927425e-08   
2024-06-20 22:09:48.451186          0.222440 -0.000034  3.047855e-04   
2024-06-20 22:09:48.466811          0.220095 -0.000044 -1.876485e-08   
2024-06-20 22:09:48.482436          0.217193 -0.000036 -3.120671e-04   
2024-06-20 22:09:48.498061          0.213721 -0.000029  1.834376e-08   

                                Abdo       SpO2  Movement Time (MT)  \
2024-06-20 22:09:48.435561  0.437458  98.084992                   0   
2024-06-20 22:09:48.451186  0.462226  98.083586                   0   
2024-06-20 22:09:48.466811  0.478490  98.084993                   0   
2024-06-20 22:09:48.482436  0.488376  98.086501                   0   
2024-06-20 22:09:48.498061  0.493507  98.084992                   0   

                            Wakefulness (W)  NREM Sleep Stage 1 (N1)  \
20

In [4]:
merged_df.columns

Index(['processed_signal', 'ECG', 'Thor', 'Abdo', 'SpO2', 'Movement Time (MT)',
       'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 'REM Sleep',
       'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)', 'Mixed Apnea',
       'Limb Movement (Left)', 'Limb Movement (Right)', 'SpO2 desaturation',
       'Hypopnea', 'PLM (Left)', 'PLM (Right)', 'SpO2 artifact',
       'Arousal (ARO SPONT)', 'Arousal (ARO PLM)', 'Arousal (ARO Limb)',
       'Arousal (ARO RES)', 'Central Apnea', 'Obstructive Apnea'],
      dtype='object')

In [5]:
# Applying the function to segment and label the dataframe with time columns
segmented_df = segment_and_label(merged_df, segment_sec=10, freq_hz=64)
segmented_df = segmented_df[~segmented_df['Dominant Sleep Stage'].isin(['No Stage'])]
segmented_df.head()

NameError: name 'segment_and_label' is not defined

In [ ]:

def segment_and_label_as_nd_cubes(df, segment_sec=10, freq_hz=64, label_type='stage', input_signals=['processed_signal', 'ECG', 'Thor', 'Abdo', 'SpO2']):
    """
    Segments the given DataFrame into n-dimensional cubes for each segment and assigns labels based on label_type.
    
    Parameters:
    - df (pandas.DataFrame): The DataFrame containing the data to be segmented.
    - segment_sec (int): The duration of each segment in seconds. Default is 10.
    - freq_hz (int): The frequency of the data in Hz. Default is 64.
    - label_type (str): The type of label to attach ('stage' for sleep stage, 'event' for sleep event). Default is 'stage'.
    
    Returns:
    - X (numpy.ndarray): A 3D array (n-dimensional cube) representing the data for each segment.
    - y (numpy.ndarray): An array of labels for each segment.
    """
    # Number of data points per segment
    segment_size = segment_sec * freq_hz
    
    # Define columns for sleep stages and sleep events
    sleep_stages = ['Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 'REM Sleep', 
                    'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)']
    sleep_events = ['Mixed Apnea', 'Limb Movement (Left)', 'Limb Movement (Right)', 'SpO2 desaturation', 
                    'Hypopnea', 'PLM (Left)', 'PLM (Right)', 'SpO2 artifact', 'Arousal (ARO SPONT)', 
                    'Arousal (ARO PLM)', 'Arousal (ARO Limb)', 'Arousal (ARO RES)', 'Central Apnea', 
                    'Obstructive Apnea']
    
    # Initialize lists to store the segmented data and labels
    X = []
    y = []

    # Loop over the DataFrame in chunks of segment_size
    for start in range(0, len(df), segment_size):
        # Get current segment
        segment = df.iloc[start:start + segment_size]
        
        # Check if segment size is not less than required (especially for the last segment)
        if len(segment) < segment_size:
            continue
        
        # Create a 2D array (segment_size x number_of_features) for the current segment
        segment_array = segment[input_signals].values
        
        # Append the array to X
        X.append(segment_array)
        
        # Determine the label based on label_type
        if label_type == 'stage':
            # Determine the dominant sleep stage (most frequent one)
            sleep_stage_counts = segment[sleep_stages].sum()
            dominant_sleep_stage = sleep_stage_counts.idxmax() if sleep_stage_counts.max() > 0 else 'No Stage'
            y.append(dominant_sleep_stage)
        elif label_type == 'event':
            # Determine the dominant sleep event (most frequent one)
            sleep_event_counts = segment[sleep_events].sum()
            dominant_sleep_event = sleep_event_counts.idxmax() if sleep_event_counts.max() > 0 else 'No Event'
            y.append(dominant_sleep_event)
        else:
            raise ValueError("Invalid label_type. Expected 'stage' or 'event'.")
    
    # Convert X and y to numpy arrays
    X = np.array(X)
    y = np.array(y)
    
    return X, y

# Example usage:
# Define columns for continuous data
continuous_columns = ['processed_signal', 'ECG', 'Thor', 'Abdo', 'SpO2']

# Apply the function to your DataFrame with 'stage' label
X_stage, y_stage = segment_and_label_as_nd_cubes(merged_df, input_signals = continuous_columns, label_type='stage')

# Apply the function to your DataFrame with 'event' label
X_event, y_event = segment_and_label_as_nd_cubes(merged_df, input_signals = continuous_columns, label_type='event')

print("Shape of X_stage:", X_stage.shape)  # Should be (number of segments, segment_size, number of features)
print("Shape of y_stage:", y_stage.shape)  # Should be (number of segments,)
print("Example label for 'stage' (y_stage[0]):", y_stage[0])

print("Shape of X_event:", X_event.shape)  # Should be (number of segments, segment_size, number of features)
print("Shape of y_event:", y_event.shape)  # Should be (number of segments,)
print("Example label for 'event' (y_event[0]):", y_event[0])


# Test 6: ML Model Training

## Random Forest

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

### Use Mean value for continuous data

In [ ]:
# Load the segmented DataFrame
df = segmented_df.copy()

# Encode target variable
label_encoder = LabelEncoder()
df['Dominant Sleep Stage'] = label_encoder.fit_transform(df['Dominant Sleep Stage'])

# Define features (X) and target (y)
features = ['processed_signal', 'ECG', 'Thor', 'Abdo', 'SpO2']
X = df[features]
y = df['Dominant Sleep Stage']

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# Train the model on the training data
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Classification report and confusion matrix
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Visualize the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


### Use n-D structure for continuous data

In [ ]:
# Define columns for continuous data
continuous_columns = ['processed_signal', 'SpO2']

# Apply the function to your DataFrame with 'stage' label
X_stage, y_stage = segment_and_label_as_nd_cubes(merged_df, input_signals = continuous_columns, label_type='stage')

In [ ]:
# Continue with the training and testing process using Random Forest
# Encode target variable
label_encoder = LabelEncoder()
y_stage_encoded = label_encoder.fit_transform(y_stage)

# Flatten X_stage for input into RandomForest (reshaping for (samples, features))
n_samples, n_timesteps, n_features = X_stage.shape
X_stage_flattened = X_stage.reshape(n_samples, n_timesteps * n_features)

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X_stage_flattened, y_stage_encoded, test_size=0.2, random_state=42, stratify=y_stage_encoded)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Get the unique classes in y_test to match with target_names
unique_classes_in_test = np.unique(y_test)

# Create a mapping from label indices to label names for the test data only
test_class_names = label_encoder.inverse_transform(unique_classes_in_test)

# Generate classification report with dynamically matched target names
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=test_class_names))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Visualize the confusion matrix with the correct labels
plt.figure(figsize=(10, 7))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=test_class_names, yticklabels=test_class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Define columns for continuous data
continuous_columns = ['processed_signal', 'Thor']

# Apply the function to your DataFrame with 'stage' label
X_stage, y_stage = segment_and_label_as_nd_cubes(merged_df, input_signals = continuous_columns, label_type='stage')

In [ ]:
# Continue with the training and testing process using Random Forest
# Encode target variable
label_encoder = LabelEncoder()
y_stage_encoded = label_encoder.fit_transform(y_stage)

# Flatten X_stage for input into RandomForest (reshaping for (samples, features))
n_samples, n_timesteps, n_features = X_stage.shape
X_stage_flattened = X_stage.reshape(n_samples, n_timesteps * n_features)

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X_stage_flattened, y_stage_encoded, test_size=0.2, random_state=42, stratify=y_stage_encoded)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Get the unique classes in y_test to match with target_names
unique_classes_in_test = np.unique(y_test)

# Create a mapping from label indices to label names for the test data only
test_class_names = label_encoder.inverse_transform(unique_classes_in_test)

# Generate classification report with dynamically matched target names
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=test_class_names))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Visualize the confusion matrix with the correct labels
plt.figure(figsize=(10, 7))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=test_class_names, yticklabels=test_class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Define columns for continuous data
continuous_columns = ['processed_signal']

# Apply the function to your DataFrame with 'stage' label
X_stage, y_stage = segment_and_label_as_nd_cubes(merged_df, input_signals = continuous_columns, label_type='stage')

In [ ]:
# Continue with the training and testing process using Random Forest
# Encode target variable
label_encoder = LabelEncoder()
y_stage_encoded = label_encoder.fit_transform(y_stage)

# Flatten X_stage for input into RandomForest (reshaping for (samples, features))
n_samples, n_timesteps, n_features = X_stage.shape
X_stage_flattened = X_stage.reshape(n_samples, n_timesteps * n_features)

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X_stage_flattened, y_stage_encoded, test_size=0.2, random_state=42, stratify=y_stage_encoded)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Get the unique classes in y_test to match with target_names
unique_classes_in_test = np.unique(y_test)

# Create a mapping from label indices to label names for the test data only
test_class_names = label_encoder.inverse_transform(unique_classes_in_test)

# Generate classification report with dynamically matched target names
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=test_class_names))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Visualize the confusion matrix with the correct labels
plt.figure(figsize=(10, 7))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=test_class_names, yticklabels=test_class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

## CNN

### 1D CNN

In [ ]:
# Define columns for continuous data
continuous_columns = ['processed_signal']

# Apply the function to your DataFrame with 'stage' label
X_stage, y_stage = segment_and_label_as_nd_cubes(merged_df, input_signals = continuous_columns, label_type='stage')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# Assuming X_stage and y_stage are defined as in your previous steps
# Encode target variable
label_encoder = LabelEncoder()
y_stage_encoded = label_encoder.fit_transform(y_stage)

# Convert X_stage to 3D format for CNN input
# X_stage = np.expand_dims(X_stage, axis=-1)  # Add a channel dimension for CNN input

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X_stage, y_stage_encoded, test_size=0.2, random_state=42, stratify=y_stage_encoded)

# Build a basic CNN model
cnn_model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Dropout(0.25),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(y_stage_encoded)), activation='softmax')  # Output layer for classification
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = np.argmax(cnn_model.predict(X_test), axis=-1)

# Get the unique classes in y_test to match with target_names
unique_classes_in_test = np.unique(y_test)

# Create a mapping from label indices to label names for the test data only
test_class_names = label_encoder.inverse_transform(unique_classes_in_test)

# Generate classification report with dynamically matched target names
print("CNN Classification Report:\n", classification_report(y_test, y_pred, target_names=test_class_names))
print("CNN Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Visualize the confusion matrix with the correct labels
plt.figure(figsize=(10, 7))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=test_class_names, yticklabels=test_class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Generate classification report with dynamically matched target names
print("CNN Classification Report:\n", classification_report(y_test, y_pred, target_names=test_class_names))
print("CNN Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

### 2D CNN

In [ ]:
# Define columns for continuous data
continuous_columns = ['processed_signal', 'Thor', 'ECG']

# Apply the function to your DataFrame with 'stage' label
X_stage, y_stage = segment_and_label_as_nd_cubes(merged_df, input_signals = continuous_columns, label_type='stage')

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Assuming X_stage and y_stage are defined as in your previous steps
# Encode target variable
label_encoder = LabelEncoder()
y_stage_encoded = label_encoder.fit_transform(y_stage)

# Reshape X_stage to 4D for 2D CNN input (samples, height, width, channels)
# Here, we use the second dimension (time steps) as "height" and the third dimension (features) as "width"
# X_stage_reshaped = np.expand_dims(X_stage, axis=-1)  # Add a channel dimension for CNN input

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X_stage, y_stage_encoded, test_size=0.2, random_state=42, stratify=y_stage_encoded)

# Build a basic 2D CNN model
cnn_model = Sequential([
    Conv2D(filters=32, kernel_size=(2, 2), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(y_stage_encoded)), activation='softmax')  # Output layer for classification
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = np.argmax(cnn_model.predict(X_test), axis=-1)

# Get the unique classes in y_test to match with target_names
unique_classes_in_test = np.unique(y_test)

# Create a mapping from label indices to label names for the test data only
test_class_names = label_encoder.inverse_transform(unique_classes_in_test)

# Generate classification report with dynamically matched target names
print("CNN Classification Report:\n", classification_report(y_test, y_pred, target_names=test_class_names))
print("CNN Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Visualize the confusion matrix with the correct labels
plt.figure(figsize=(10, 7))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=test_class_names, yticklabels=test_class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Generate classification report with dynamically matched target names
print("CNN Classification Report:\n", classification_report(y_test, y_pred, target_names=test_class_names))
print("CNN Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

### RNN

In [ ]:
# Define columns for continuous data
continuous_columns = ['processed_signal', 'Thor', 'ECG', 'SpO2']

# Apply the function to your DataFrame with 'stage' label
X_stage, y_stage = segment_and_label_as_nd_cubes(merged_df, input_signals = continuous_columns, label_type='stage')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming X_stage and y_stage are defined as in your previous steps
# Encode target variable
label_encoder = LabelEncoder()
y_stage_encoded = label_encoder.fit_transform(y_stage)

# Reshape X_stage to ensure it's 3D for LSTM input (samples, timesteps, features)
# Here, X_stage should already be 3D if prepared as time-series data (samples, time steps, features)
# No need for reshaping if X_stage is already correctly shaped

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X_stage, y_stage_encoded, test_size=0.2, random_state=42, stratify=y_stage_encoded)

# Build a basic LSTM RNN model
rnn_model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),  # LSTM layer with 64 units
    Dropout(0.2),
    LSTM(32, return_sequences=False),  # LSTM layer with 32 units
    Dropout(0.2),
    Dense(len(np.unique(y_stage_encoded)), activation='softmax')  # Output layer for classification
])

# Compile the model
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
rnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = np.argmax(rnn_model.predict(X_test), axis=-1)

# Create a mapping from label indices to label names for the test data only
test_class_names = label_encoder.inverse_transform(np.unique(y_test))

# Generate classification report with dynamically matched target names
print("RNN Classification Report:\n", classification_report(y_test, y_pred, target_names=test_class_names))
print("RNN Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Visualize the confusion matrix with the correct labels
plt.figure(figsize=(10, 7))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=test_class_names, yticklabels=test_class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import classification_report, confusion_matrix
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# # Assuming X_stage and y_stage are defined as in your previous steps
# # Encode target variable
# label_encoder = LabelEncoder()
# y_stage_encoded = label_encoder.fit_transform(y_stage)

# # Split the data into training and testing sets (80-20 split)
# X_train, X_test, y_train, y_test = train_test_split(X_stage, y_stage_encoded, test_size=0.2, random_state=42, stratify=y_stage_encoded)

# # Define the model creation function
# def create_model(units=64, dropout_rate=0.2):
#     model = Sequential([
#         LSTM(units, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
#         Dropout(dropout_rate),
#         LSTM(units // 2, return_sequences=False),
#         Dropout(dropout_rate),
#         Dense(len(np.unique(y_stage_encoded)), activation='softmax')  # Output layer for classification
#     ])
    
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#     return model

# # Wrap the model with KerasClassifier for scikit-learn
# model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# # Define the grid of hyperparameters to search
# param_grid = {
#     'units': [32, 64, 128],  # Number of LSTM units
#     'dropout_rate': [0.2, 0.3, 0.4],  # Dropout rates
#     'epochs': [10, 20],  # Number of epochs
#     'batch_size': [32, 64]  # Batch sizes
# }

# # Set up GridSearchCV
# grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# # Run the grid search
# grid_result = grid.fit(X_train, y_train)

# # Print the best score and the best parameters found
# print(f"Best Accuracy: {grid_result.best_score_:.4f}")
# print("Best Parameters:", grid_result.best_params_)

# # Use the best estimator to make predictions
# best_model = grid_result.best_estimator_
# y_pred = best_model.predict(X_test)

# # Create a mapping from label indices to label names for the test data only
# test_class_names = label_encoder.inverse_transform(np.unique(y_test))

# # Generate classification report with dynamically matched target names
# print("RNN Classification Report:\n", classification_report(y_test, y_pred, target_names=test_class_names))
# print("RNN Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
# # Print the best score and the best parameters found
# print(f"Best Accuracy: {grid_result.best_score_:.4f}")
# print("Best Parameters:", grid_result.best_params_)

# # Use the best estimator to make predictions
# best_model = grid_result.best_estimator_
# y_pred = best_model.predict(X_test)

# # Create a mapping from label indices to label names for the test data only
# test_class_names = label_encoder.inverse_transform(np.unique(y_test))

# # Generate classification report with dynamically matched target names
# print("RNN Classification Report:\n", classification_report(y_test, y_pred, target_names=test_class_names))
# print("RNN Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
# grid_result
# # {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 20, 'units': 128}

## Transformer

In [ ]:
# Define columns for continuous data
continuous_columns = ['processed_signal', 'Thor', 'ECG', 'SpO2']

# Apply the function to your DataFrame with 'stage' label
X_stage, y_stage = segment_and_label_as_nd_cubes(merged_df, input_signals = continuous_columns, label_type='stage')

In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, Embedding, LayerNormalization, Dropout
# from tensorflow.keras.layers import MultiHeadAttention, Add, GlobalAveragePooling1D
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import classification_report, confusion_matrix
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Assuming X_stage and y_stage are defined and preprocessed as in your previous steps
# # Encode target variable
# label_encoder = LabelEncoder()
# y_stage_encoded = label_encoder.fit_transform(y_stage)

# # Reshape X_stage to ensure it's 3D (samples, timesteps, features)
# # X_stage = np.expand_dims(X_stage, axis=-1)  # Add a channel dimension if not already present

# # Split the data into training and testing sets (80-20 split)
# X_train, X_test, y_train, y_test = train_test_split(X_stage, y_stage_encoded, test_size=0.2, random_state=42, stratify=y_stage_encoded)

# # Define the Transformer model
# def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
#     # Normalization and Attention
#     x = LayerNormalization(epsilon=1e-6)(inputs)
#     x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
#     x = Dropout(dropout)(x)
#     res = Add()([x, inputs])

#     # Feed Forward Part
#     x = LayerNormalization(epsilon=1e-6)(res)
#     x = Dense(ff_dim, activation="relu")(x)
#     x = Dropout(dropout)(x)
#     x = Dense(inputs.shape[-1])(x)
#     return Add()([x, res])

# def build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
#     inputs = Input(shape=input_shape)
#     x = inputs
#     for _ in range(num_transformer_blocks):
#         x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

#     x = GlobalAveragePooling1D(data_format="channels_first")(x)
#     for dim in mlp_units:
#         x = Dense(dim, activation="relu")(x)
#         x = Dropout(mlp_dropout)(x)
#     outputs = Dense(len(np.unique(y_stage_encoded)), activation="softmax")(x)
#     return Model(inputs, outputs)

# input_shape = X_train.shape[1:]
# model = build_transformer_model(
#     input_shape,
#     head_size=256,
#     num_heads=4,
#     ff_dim=4,
#     num_transformer_blocks=4,
#     mlp_units=[128],
#     dropout=0.25,
#     mlp_dropout=0.4,
# )

# model.compile(
#     loss="sparse_categorical_crossentropy",
#     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
#     metrics=["sparse_categorical_accuracy"],
# )

# # Train the model
# model.fit(
#     X_train,
#     y_train,
#     validation_split=0.2,
#     epochs=20,
#     batch_size=32,
# )

# # Evaluate the model
# y_pred = np.argmax(model.predict(X_test), axis=-1)

# # Create a mapping from label indices to label names for the test data only
# test_class_names = label_encoder.inverse_transform(np.unique(y_test))

# # Generate classification report with dynamically matched target names
# print("Transformer Classification Report:\n", classification_report(y_test, y_pred, target_names=test_class_names))
# print("Transformer Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# # Visualize the confusion matrix with the correct labels
# plt.figure(figsize=(10, 7))
# sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=test_class_names, yticklabels=test_class_names)
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.title('Confusion Matrix')
# plt.show()


In [ ]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available. Here are the details:")
    print(" - GPU Name:", torch.cuda.get_device_name(0))
    print(" - Number of GPUs:", torch.cuda.device_count())
    print(" - Current GPU:", torch.cuda.current_device())
else:
    print("GPU is not available. Please check your CUDA installation.")

# Perform a simple computation to test the GPU
a = torch.randn(1000, 1000, device='cuda')
b = torch.randn(1000, 1000, device='cuda')

print("Performing a matrix multiplication to test the GPU...")

import time
start_time = time.time()

c = torch.matmul(a, b)

print("Matrix multiplication completed.")
print("Time taken: {:.3f} seconds".format(time.time() - start_time))
print("Device used for computation:", c.device)


In [ ]:
import torch
print(torch.backends.cudnn.version())


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Encode target variable
label_encoder = LabelEncoder()
y_stage_encoded = label_encoder.fit_transform(y_stage)

# Convert data to PyTorch tensors
X_stage_tensor = torch.tensor(X_stage, dtype=torch.float32).to(device)
y_stage_tensor = torch.tensor(y_stage_encoded, dtype=torch.long).to(device)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_stage_tensor, y_stage_tensor, test_size=0.2, random_state=42, stratify=y_stage_encoded)

# Create PyTorch datasets and loaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the Transformer model
class TransformerClassifier(nn.Module):
    def __init__(self, input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0.25, mlp_dropout=0.4):
        super(TransformerClassifier, self).__init__()
        self.num_transformer_blocks = num_transformer_blocks
        self.attention_heads = nn.ModuleList([
            nn.MultiheadAttention(embed_dim=head_size, num_heads=num_heads, dropout=dropout) for _ in range(num_transformer_blocks)
        ])
        self.norm_layers = nn.ModuleList([nn.LayerNorm(head_size) for _ in range(num_transformer_blocks)])
        self.ff_layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(head_size, ff_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(ff_dim, head_size)
            ) for _ in range(num_transformer_blocks)
        ])
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.mlp = nn.Sequential(
            nn.Linear(head_size, mlp_units[0]),
            nn.ReLU(),
            nn.Dropout(mlp_dropout),
            nn.Linear(mlp_units[0], len(np.unique(y_stage_encoded)))
        )

    def forward(self, x):
        for i in range(self.num_transformer_blocks):
            attn_output, _ = self.attention_heads[i](x, x, x)
            x = self.norm_layers[i](x + attn_output)
            ff_output = self.ff_layers[i](x)
            x = self.norm_layers[i](x + ff_output)
        x = self.pool(x.transpose(1, 2)).squeeze(-1)
        x = self.mlp(x)
        return x

input_shape = X_train.shape[1:]
model = TransformerClassifier(
    input_shape=input_shape[1],  # Assuming input_shape is (timesteps, features)
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    dropout=0.25,
    mlp_dropout=0.4,
).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

# Evaluation function
def evaluate_model(model, test_loader, device):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    return y_true, y_pred

# Train the model for 20 epochs
epochs = 20
for epoch in range(epochs):
    train_loss = train_model(model, train_loader, criterion, optimizer, device)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}')

# Evaluate the model
y_true, y_pred = evaluate_model(model, test_loader, device)

# Create a mapping from label indices to label names for the test data only
test_class_names = label_encoder.inverse_transform(np.unique(y_test.cpu().numpy()))

# Generate classification report with dynamically matched target names
print("Transformer Classification Report:\n", classification_report(y_true, y_pred, target_names=test_class_names))
print("Transformer Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

# Visualize the confusion matrix with the correct labels
plt.figure(figsize=(10, 7))
sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=test_class_names, yticklabels=test_class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
